# Webscraping

Im foglenden Notebook wird die Website https://issuu.com/fhnw/docs/modultabelle_20maschinenbau gescrpat, damit wir die Modulinformationen zu dem Studiengang Maschinenbau erhalten.

In [8]:
import pdfplumber
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re


In [9]:
driver = webdriver.Chrome()  # oder z.B. webdriver.Firefox()
driver.get("https://issuu.com/fhnw/docs/modultabelle_20maschinenbau")

soup = BeautifulSoup(driver.page_source, "html.parser")
iframe = soup.find("iframe")
if iframe:
    src_url = iframe["src"]
    print("PDF/View URL:", src_url)
else:
    print("Kein iframe gefunden!")

driver.quit()

PDF/View URL: https://issuu.com/rd4?p=1&d=modultabelle_20maschinenbau&u=fhnw


In [10]:
pdf_path = "../doc/Modultabelle Maschinenbau_HS2025_updated.pdf"
data_rows = []

# Hilfsfunktion zum Erkennen von Modulgruppen (kann an das Dokument angepasst werden)
def is_modulgruppe(text):
    if not text:
        return False
    # Beispiel: enthält ":" und kein Wort "Minimum" oder "alle"
    text = text.lower()
    return (":" in text) and ("minimum" not in text) and ("alle" not in text)

# Hilfsfunktion zum Erkennen gültiger Moduldatenzeilen
def is_gültige_modulzeile(row):
    if not row or len(row) < 2:
        return False
    # Modulname mindestens vorhanden und kein einleitender Text wie "Minimum" etc.
    modul = row[0]
    if modul is None:
        return False
    modul_lower = modul.lower()
    if "minimum" in modul_lower or "alle" in modul_lower:
        return False
    if modul.strip() == "":
        return False
    return True

with pdfplumber.open(pdf_path) as pdf:
    current_modulgruppe = ""
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                # Prüfen ob Modulgruppe
                if row[0] and is_modulgruppe(row[0]):
                    current_modulgruppe = row[0].strip()
                    continue
                # Überspringe Header oder unerwünschte Zeilen
                if not is_gültige_modulzeile(row):
                    continue
                # Extrahiere Moduleinträge
                modul = row[0] if row[0] else ""
                kuerzel = row[1] if len(row) > 1 and row[1] else ""
                voraussetzung = row[4] if len(row) > 4 and row[4] else ""
                
                # Semesterfindung (Spalten nach Nr. 4 durchsuchen)
                semester = ""
                for i in range(5, len(row)):
                    if row[i] and row[i].strip():
                        semester = str(i - 4)
                        break

                data_rows.append({
                    "Modulgruppe": current_modulgruppe,
                    "Modul": modul.strip(),
                    "Kürzel": kuerzel.strip(),
                    "Voraussetzung": voraussetzung.strip(),
                    "Semester": semester
                })

df = pd.DataFrame(data_rows)

In [11]:
df

,Modulgruppe,Modul,Kürzel,Voraussetzung,Semester
0,Grundlagen: Mathematik 1,Lineare Algebra 1,lalg1,,11
1,Grundlagen: Mathematik 1,Lineare Algebra 2,lalg2,,4
2,Grundlagen: Mathematik 1,Informatik (M),infM,,4
3,Grundlagen: Mathematik 1,Wahrscheinlichkeitstheorie und Statistik,wst,,4
4,Grundlagen: Mathematik 1,Datenanalyse (Machine Learning),dan,,4
...,...,...,...,...,...
87,Projekte: Maschinenbau,Kontext,min. Anzahl Module,,
88,Projekte: Maschinenbau,Kommunikation,2,,
89,Projekte: Maschinenbau,Englisch,4,,
90,Projekte: Maschinenbau,Betriebswirtschaftslehre,2,,


In [13]:
pdf_path = "../doc/Modultabelle Maschinenbau_HS2025_updated.pdf"

data_rows = []

def is_modulgruppe(text):
    if not text:
        return False
    text = text.lower()
    return (":" in text) and ("minimum" not in text) and ("alle" not in text)

def is_gültige_modulzeile(row):
    if not row or len(row) < 2:
        return False
    modul = row[0]
    if modul is None:
        return False
    modul_lower = modul.lower()
    if "minimum" in modul_lower or "alle" in modul_lower:
        return False
    if modul.strip() == "":
        return False
    return True

with pdfplumber.open(pdf_path) as pdf:
    current_modulgruppe = ""
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                if row[0] and is_modulgruppe(row[0]):
                    current_modulgruppe = row[0].strip()
                    continue
                if not is_gültige_modulzeile(row):
                    continue
                
                modul = row[0] if row[0] else ""
                kuerzel = row[1] if len(row) > 1 and row[1] else ""
                
                # fachliche Voraussetzung an korrekter Stelle (hier z.B. Spalte 3 oder je nach Tabelle anpassen)
                fachliche_voraussetzung = ""
                if len(row) > 3 and row[3]:
                    fachliche_voraussetzung = row[3].strip()
                
                # Semesterbestimmung: angenommen ab Spalte 5 bis Ende sind Semester-Spalten (Index 4+)
                semester = ""
                for i in range(5, len(row)):
                    val = row[i]
                    if val and val.strip():
                        # Das ist die markierte Semester-Spalte (Parsen je nach PDF-Format anpassen)
                        semester = str(i - 4)  # z.B. erste markierte Spalte als Semester
                        break
                
                data_rows.append({
                    "Modulgruppe": current_modulgruppe,
                    "Modul": modul.strip(),
                    "Kürzel": kuerzel.strip(),
                    "Voraussetzung": fachliche_voraussetzung,
                    "Semester": semester
                })

df = pd.DataFrame(data_rows)


In [14]:
df

,Modulgruppe,Modul,Kürzel,Voraussetzung,Semester
0,Grundlagen: Mathematik 1,Lineare Algebra 1,lalg1,,11
1,Grundlagen: Mathematik 1,Lineare Algebra 2,lalg2,,4
2,Grundlagen: Mathematik 1,Informatik (M),infM,,4
3,Grundlagen: Mathematik 1,Wahrscheinlichkeitstheorie und Statistik,wst,,4
4,Grundlagen: Mathematik 1,Datenanalyse (Machine Learning),dan,,4
...,...,...,...,...,...
87,Projekte: Maschinenbau,Kontext,min. Anzahl Module,,
88,Projekte: Maschinenbau,Kommunikation,2,,
89,Projekte: Maschinenbau,Englisch,4,,
90,Projekte: Maschinenbau,Betriebswirtschaftslehre,2,,


In [16]:
pdf_path = "../doc/Modultabelle Maschinenbau_HS2025_updated.pdf"
data_rows = []

def is_modulgruppe(text):
    if not text: 
        return False
    text = text.lower()
    return (":" in text) and ("minimum" not in text) and ("alle" not in text)

def is_gültige_modulzeile(row):
    if not row or len(row) < 2: 
        return False
    modul = row[0]
    if modul is None: 
        return False
    modul_lower = modul.lower()
    if "minimum" in modul_lower or "alle" in modul_lower: 
        return False
    if modul.strip() == "": 
        return False
    return True

with pdfplumber.open(pdf_path) as pdf:
    current_modulgruppe = ""
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                if row[0] and is_modulgruppe(row[0]):
                    current_modulgruppe = row[0].strip()
                    continue
                if not is_gültige_modulzeile(row):
                    continue
                
                modul = row[0].strip() if row[0] else ""
                kuerzel = row[1].strip() if len(row) > 1 and row[1] else ""
                
                # fachl. Voraussetzung gewöhnlich an Spalte 3 (Index 3) oder 4
                voraussetzung = ""
                if len(row) > 3 and row[3]:
                    voraussetzung = row[3].strip()
                
                # Semester finden: Ab Spalte 5 (Index 4) bis Ende; erste gefüllte Spalte als Semester
                semester = ""
                semester_cols_start = 5
                for i in range(semester_cols_start, len(row)):
                    val = str(row[i]).strip() if row[i] else ""
                    # Prüfe ob Zelle gefüllt ist (typisch: 'x', '1', '✓' o.ä. oder Nicht-Leer)
                    if val != "":
                        semester = str(i - semester_cols_start + 1)
                        break
                
                data_rows.append({
                    "Modulgruppe": current_modulgruppe,
                    "Modul": modul,
                    "Kürzel": kuerzel,
                    "Voraussetzung": voraussetzung,
                    "Semester": semester
                })

df = pd.DataFrame(data_rows)
df.to_excel('../data/data.xlsx')


In [17]:
data = pd.read_excel('../data/data.xlsx')
data.to_csv('../data/data.csv')

## Daten für Gephi

In [25]:

# 1. CSV laden
data = pd.read_csv('../data/data.csv', sep=',', dtype=str)

print(f"Gefundene Module: {len(data)}")
print(data.head())

# 2. Alle einzigartigen Kuerzel sammeln (Nodes)
alle_kuerzel = set()
for idx, row in data.iterrows():
    kuerzel = str(row['Kürzel']).strip()
    voraus = str(row['Voraussetzung']).strip()  # <- FIX: str() vor strip()
    
    alle_kuerzel.add(kuerzel)
    
    if voraus != 'nan' and voraus != '' and voraus != 'None':
        voraus_list = [v.strip() for v in voraus.split(',')]
        alle_kuerzel.update(voraus_list)

print(f"\nAlle einzigartige Knoten: {len(alle_kuerzel)}")

# 3. Nodes DataFrame bauen
nodes_df = pd.DataFrame({
    'Id': list(alle_kuerzel),
    'Label': list(alle_kuerzel)
})

# Nodes mit Zusatzinfo anreichern
for idx, row in data.iterrows():
    kuerzel = str(row['Kürzel']).strip()
    if kuerzel in nodes_df['Id'].values:
        mod_idx = nodes_df[nodes_df['Id'] == kuerzel].index[0]
        nodes_df.at[mod_idx, 'Name'] = str(row['Modul'])
        nodes_df.at[mod_idx, 'Gruppe'] = str(row['Modulgruppe'])
        nodes_df.at[mod_idx, 'Semester'] = str(row['Semester'])

nodes_df.to_csv('../data/nodes.csv', index=False)
print("\nnodes.csv erstellt")

# 4. Edges DataFrame bauen
edges = []
for idx, row in data.iterrows():
    kuerzel = str(row['Kürzel']).strip()
    voraus = str(row['Voraussetzung']).strip()
    
    if voraus != 'nan' and voraus != '' and voraus != 'None':
        voraus_list = [v.strip() for v in voraus.split(',')]
        for v in voraus_list:
            if v in alle_kuerzel:
                edges.append({
                    'Source': v,
                    'Target': kuerzel,
                    'Type': 'Directed',
                    'Label': 'Voraussetzung'
                })

edges_df = pd.DataFrame(edges)
edges_df.to_csv('../data/edges.csv', index=False)

print(f"edges.csv erstellt mit {len(edges_df)} Kanten")
print("\nDateien bereit fuer Gephi!")
print("nodes.csv:", nodes_df.head())
print("edges.csv:", edges_df.head())


Gefundene Module: 92
  Unnamed: 0.1 Unnamed: 0               Modulgruppe  \
0            0          0  Grundlagen: Mathematik 1   
1            1          1  Grundlagen: Mathematik 1   
2            2          2  Grundlagen: Mathematik 1   
3            3          3  Grundlagen: Mathematik 1   
4            4          4  Grundlagen: Mathematik 1   

                                      Modul Kürzel Voraussetzung Semester  
0                         Lineare Algebra 1  lalg1           NaN     11.0  
1                         Lineare Algebra 2  lalg2           NaN      4.0  
2                            Informatik (M)   infM           NaN      4.0  
3  Wahrscheinlichkeitstheorie und Statistik    wst           NaN      4.0  
4           Datenanalyse (Machine Learning)    dan           NaN      4.0  

Alle einzigartige Knoten: 92

nodes.csv erstellt
edges.csv erstellt mit 12 Kanten

Dateien bereit fuer Gephi!
nodes.csv:       Id  Label                 Name                                 G